<a href="https://colab.research.google.com/github/julianacmsantos/recipes-project/blob/main/2_fase_recipes_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importar bibliotecas
!pip install -q sentence-transformers faiss-cpu pandas tqdm

from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm
import os

In [3]:
csv_path = "/content/drive/MyDrive/Projetos/recipes_project/recipenlg_reduzido.csv"
output_dir = "/content/drive/MyDrive/Projetos/recipes_project/embeddings_index"
os.makedirs(output_dir, exist_ok=True)

print("Carregando CSV reduzido...")
df = pd.read_csv(csv_path)
print(f"Linhas carregadas: {len(df)}")

Carregando CSV reduzido...
Linhas carregadas: 100000


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# garantir que ner está em formato de lista ou texto; criar ner_text
def ner_to_text(x):
    # se já for lista (pandas pode guardar listas), junte; se for string, tente avaliar; senão str()
    if isinstance(x, list):
        return " ".join(x)
    if isinstance(x, str):
        # evitar erros: remover colchetes se já for string serializada
        # assumimos que a conversão já foi feita no preprocessing; apenas join simples
        # se for string tipo "['a','b']", podemos tentar ast, mas assumimos ok.
        return x
    return str(x)

df['ner_text'] = df['ner'].apply(ner_to_text)
# Opcional: concatenar título para mais contexto
df['embed_text'] = df['title'].astype(str) + " | " + df['ner_text'].astype(str)

print("Modelo de embedding: sentence-transformers/all-MiniLM-L6-v2")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Modelo de embedding: sentence-transformers/all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# gerar embeddings em batch
batch_size = 512
texts = df['embed_text'].tolist()
n = len(texts)
dim = 384  # dimensão do all-MiniLM-L6-v2

embeddings = np.zeros((n, dim), dtype='float32')

print("Gerando embeddings (isso pode levar alguns minutos)...")
for i in tqdm(range(0, n, batch_size)):
    batch_texts = texts[i:i+batch_size]
    emb = model.encode(batch_texts, convert_to_numpy=True, show_progress_bar=False)
    embeddings[i:i+len(emb)] = emb.astype('float32')

# normalizar embeddings para usar IndexFlatIP (produto interno ~ similaridade coseno)
faiss.normalize_L2(embeddings)

# salvar embeddings (opcional)
emb_path = os.path.join(output_dir, "embeddings.npy")
np.save(emb_path, embeddings)
print("Embeddings salvos em:", emb_path)

Gerando embeddings (isso pode levar alguns minutos)...


100%|██████████| 196/196 [56:31<00:00, 17.31s/it]


Embeddings salvos em: /content/drive/MyDrive/Projetos/recipes_project/embeddings_index/embeddings.npy


In [7]:
# criar índice FAISS (IndexFlatIP: busca exata por produto interno)
index = faiss.IndexFlatIP(dim)
index.add(embeddings)
print("Total de vetores no índice:", index.ntotal)

# salvar índice em disco
index_path = os.path.join(output_dir, "faiss_index.index")
faiss.write_index(index, index_path)
print("Índice FAISS salvo em:", index_path)

Total de vetores no índice: 100000
Índice FAISS salvo em: /content/drive/MyDrive/Projetos/recipes_project/embeddings_index/faiss_index.index


In [9]:
# salvar metadata (id -> fields) para retorno depois
# vamos salvar CSV/JSON com id numérico (posição) e colunas úteis
metadata = df[['title', 'ingredients', 'instructions', 'link', 'ner']].copy()
metadata['id'] = metadata.index
meta_path = os.path.join(output_dir, "metadata.csv")
metadata.to_csv(meta_path, index=False)
print("Metadata salva em:", meta_path)

Metadata salva em: /content/drive/MyDrive/Projetos/recipes_project/embeddings_index/metadata.csv
